In [10]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGSMITH_PROJECT"]

'WALKER_AGENT'

# 1. WEB

In [11]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_tavily import TavilySearch

In [12]:
# 1. LLM 설정
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

# 2. TOOL 설정
tavily_search = TavilySearch(max_result=3)

# 3. PROMPT 
prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 툴만 사용해서 사실을 확인하고 간단하게 대답해줘'),
    ('human', '{input}'),
    ('placeholder', '{agent_scratchpad}') # tool 사용 결과를 담는 임시 공간
])

# 4. 단일 AGENT 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools=[tavily_search],
    prompt=prompt
)

# 5. AGENT EXECUTOR 설정
executor = AgentExecutor(
    agent=agent,
    tools=[tavily_search],
    verbose=True
)

In [5]:
result = executor.invoke({'input' : '삼성전자가 ai를 도입하면 좋을만한 사업에 대해서 자세하게 찾아줘'})
result



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '삼성전자 AI 도입 사업 분야', 'search_depth': 'advanced'}`


{'query': '삼성전자 AI 도입 사업 분야', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://news.samsung.com/kr/%EC%82%BC%EC%84%B1-openai%EC%99%80-%ED%95%A8%EA%BB%98-%EC%B0%A8%EC%84%B8%EB%8C%80-ai-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%84%BC%ED%84%B0-%ED%98%81%EC%8B%A0-%EC%A3%BC%EB%8F%84', 'title': '삼성, OpenAI와 함께 차세대 AI 데이터센터 혁신 주도', 'content': '삼성전자는 ▲메모리 기술력 ▲글로벌 반도체 업계 1위 생산 능력 ▲안정적인 글로벌 생산 거점을 기반으로 OpenAI와 같은 글로벌 AI 선도 기업과의 협력을 강화할 예정이다.\n\n또한, 미래 AI 경쟁력을 제고하기 위해 ▲대규모 R&D 투자 ▲선제적 국내외 시설 투자 ▲국내외 우수 인재 육성과 유치를 지속할 방침이다.\n\n삼성SDS, 삼성물산, 삼성중공업도 AI 사업 기회를 확대해 한국이 AI 분야에서 글로벌 인공지능 3대 강국으로 도약하는 데 기여할 계획이다.\n\n한편, 삼성은 임직원들의 기술 개발 생산성을 높이고 혁신 속도에 박차를 가하기 위해 ChatGPT 사내 확대 도입도 검토하고 있다.\n\nTAGSAI데이터센터메모리 솔루션\n\n기업뉴스 > 경영일반\n\n기업뉴스\n\n프레스센터 > 보도자료\n\n프레스센터\n\n보다 다양한 삼성전자 소식은? [...] 또한, 삼성전자는 패키징 기술, 메모리·시스템 반도체 융복합 기술 측면에도 OpenAI에 차별화된 솔루션을 제공할

{'input': '삼성전자가 ai를 도입하면 좋을만한 사업에 대해서 자세하게 찾아줘',
 'output': '삼성전자가 AI를 도입하면 좋을 사업 분야는 다음과 같습니다:\n\n1. 반도체 사업: AI를 활용해 반도체 설계, 제조 공정의 복잡성 극복, 데이터 관리 및 품질 향상에 기여할 수 있습니다. 삼성전자 DS부문 AI센터는 반도체 도메인에 특화된 AI 기술을 개발 중입니다.\n\n2. AI 데이터센터: 삼성전자는 OpenAI와 협력해 AI 데이터센터 혁신을 주도하며, 고성능·저전력 메모리 공급과 AI 데이터센터 설계, 구축, 운영에 참여하고 있습니다.\n\n3. 로봇 및 제조 자동화: 삼성전자는 로봇 전문기업 레인보우로보틱스와 협력해 협동로봇, 자율이동로봇 등을 제조 및 물류 업무 자동화에 활용하고 있습니다.\n\n4. 사내 업무 효율화: ChatGPT 등 생성형 AI를 사내 도입해 임직원들의 기술 개발 생산성을 높이고 혁신 속도를 가속화하는 방안을 검토 중입니다.\n\n이 외에도 AI를 활용한 IT 인프라 고도화, 소프트웨어 개발 및 품질 향상, 물류 로봇 경쟁력 강화 등 다양한 분야에서 AI 도입이 진행되고 있습니다.'}

In [7]:
from qdrant_client import QdrantClient

# Qdrant 서버 접속
client = QdrantClient(url="http://localhost:6333")

# 서버 정상 동작 확인
print(client.get_collections())

collections=[CollectionDescription(name='kpop_demon_hunters')]


In [ ]:
from datetime import datetime

from langchain.schema import Document
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. 텍스트 쪼개기
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(result["output"])

# 2. Document로 감싸기
today = datetime.now().strftime("%Y%m%d")# (예: '20251021')

docs = [
    Document(
        page_content=chunk,
        metadata={"source": f"samsung_external_web_{today}", "chunk_id": i}
    )
    for i, chunk in enumerate(chunks)
]

# 3. 임베딩
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 4. Qdrant에 저장
db = Qdrant.from_documents(
    documents=docs,
    embedding=embedding,
    collection_name="samsung_external_web"
)

# 2. PDF

In [21]:
from dotenv import load_dotenv

from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader

In [22]:
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

samsung_vision_2025_path = '../../raw_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf'
docs_2025 = PDFPlumberLoader(samsung_vision_2025_path).load()

print(len(docs_2025))

87


In [ ]:
from datetime import datetime

from langchain.schema import Document
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. 텍스트 쪼개기
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(docs_2025)

# 2. Document로 감싸기
today = datetime.now().strftime("%Y%m%d")# (예: '20251021')

# 3. 임베딩
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 4. Qdrant에 저장
db = Qdrant.from_documents(
    documents=docs,
    embedding=embedding,
    collection_name="samsung_external_pdf"
)